In [112]:
import pandas as pd
import os
import numpy as np

In [113]:
csv_school_path = "raw_data/schools_complete.csv"
csv_student_path ="raw_data/students_complete.csv"

In [114]:
school_raw_df = pd.read_csv(csv_school_path)
student_raw_df = pd.read_csv(csv_student_path)

In [115]:
total_school_count = len(school_raw_df)
total_student_count = len(student_raw_df)
total_budget = school_raw_df["budget"].sum()
Average_math_score = student_raw_df["math_score"].mean()
Average_reading_score = student_raw_df["reading_score"].mean()

#Passing score = 60
pass_score =60
math_pass_count = len(student_raw_df[student_raw_df["math_score"] >= pass_score])
reading_pass_count = len(student_raw_df[student_raw_df["reading_score"] >= pass_score])
math_pass_percent = math_pass_count/total_student_count *100
reading_pass_percent = reading_pass_count/total_student_count*100
average_pass_percent = ((math_pass_percent +reading_pass_percent)/2)


In [116]:
district_summary = pd.DataFrame({"Total Number of School":[total_school_count],
                               "Total Number of Student":[total_student_count],
                               "Total Budget":[total_budget],
                               "Average Math Score":[Average_math_score],
                               "Average Reading Score":[Average_reading_score],
                               "% Passing Math":[math_pass_percent],
                               "% Passing Reading":[reading_pass_percent],
                               "Overall Passing Rate":[average_pass_percent]})

district_summary

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Number of School,Total Number of Student
0,92.445749,100.0,78.985371,81.87784,96.222875,24649428,15,39170


In [117]:
## School Summary
student_raw_df = student_raw_df.rename(columns = {"school" :"school_name"})
school_raw_df = school_raw_df.rename(columns = {"name": "school_name"})
student_school_df = student_raw_df.merge(school_raw_df,on="school_name")
           

In [118]:
def math_pass_score(row):
    if row["math_score"] >= pass_score:
       status = 1
    else:
       status = 0
    return status    
student_school_df["% Passing Math"] = student_school_df.apply(math_pass_score,axis =1)
student_school_df["% Passing Reading"] = np.where(student_school_df["reading_score"]>=pass_score, 1, 0)


In [119]:
school_summary_obj = student_school_df.groupby(by='school_name')
school_student = school_summary_obj["Student ID"].count()
school_student_df = pd.DataFrame(school_student)
school_student_df = school_student_df.reset_index()                        


In [120]:
school_budget_df = school_raw_df[["school_name","type","budget"]]


In [121]:
school_summary_df = pd.merge(school_budget_df,school_student_df,on="school_name")
school_summary_df = school_summary_df.rename(columns ={"Student ID":"Total Students"})


In [122]:
school_summary_df["Per student Budget"] = school_summary_df["budget"]/school_summary_df["Total Students"]
school_summary_df = school_summary_df.sort_values("school_name")


In [123]:
avg_math_per_school = school_summary_obj["math_score"].mean()
math_percent_pass = school_summary_obj["% Passing Math"].sum()
avg_math_per_school_df = pd.DataFrame(avg_math_per_school)
math_percent_pass_df = pd.DataFrame(math_percent_pass)
avg_math_per_school_df = avg_math_per_school_df.reset_index()
math_percent_pass_df =math_percent_pass.reset_index()

avg_reading_per_school = school_summary_obj["reading_score"].mean()
reading_percent_pass = school_summary_obj["% Passing Reading"].sum()
avg_reading_per_school_df = pd.DataFrame(avg_reading_per_school)
reading_percent_pass_df=pd.DataFrame(reading_percent_pass)
avg_reading_per_school_df = avg_reading_per_school_df.reset_index()
reading_percent_pass_df = reading_percent_pass_df.reset_index()

In [124]:
school_summary_df = pd.merge(school_summary_df,avg_math_per_school_df,on="school_name")

school_summary_df = pd.merge(school_summary_df,avg_reading_per_school_df,on="school_name")

school_summary_df =pd.merge(school_summary_df,math_percent_pass_df,on="school_name")
school_summary_df = pd.merge(school_summary_df,reading_percent_pass_df,on ="school_name")

school_summary_df["% Passing Math"] = (school_summary_df["% Passing Math"]/school_summary_df["Total Students"])*100
school_summary_df["% Passing Reading"] =(school_summary_df["% Passing Reading"]/school_summary_df["Total Students"])*100

school_summary_df["Overall Passing"]=(school_summary_df["% Passing Math"]+school_summary_df["% Passing Reading"])/2


In [125]:
school_summary_df = school_summary_df.rename(columns = {"school_name":"School Name","reading_score":"Average Reading Score",
                                                        "math_score":"Average Math Score","type":"School Type","budget":"Total School Budget"
                                                    })


In [126]:
school_summary_df = school_summary_df.sort_values("Overall Passing",ascending=False)

school_summary_df.head(10)

,School Name,School Type,Total School Budget,Total Students,Per student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
1,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,100.000000,100.0,100.000000
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,100.000000,100.0,100.000000
6,Holden High School,Charter,248087,427,581.0,83.803279,83.814988,100.000000,100.0,100.000000
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,100.000000,100.0,100.000000
11,Shelton High School,Charter,1056600,1761,600.0,83.359455,83.725724,100.000000,100.0,100.000000
12,Thomas High School,Charter,1043130,1635,638.0,83.418349,83.848930,100.000000,100.0,100.000000
13,Wilson High School,Charter,1319574,2283,578.0,83.274201,83.989488,100.000000,100.0,100.000000
14,Wright High School,Charter,1049400,1800,583.0,83.682222,83.955000,100.000000,100.0,100.000000
0,Bailey High School,District,3124928,4976,628.0,77.048432,81.033963,89.529743,100.0,94.764871
3,Ford High School,District,1763916,2739,644.0,77.102592,80.746258,89.302665,100.0,94.651333


In [127]:
school_summary_top_five_df = school_summary_df.head(5)
school_summary_bottom_five_df = school_summary_df.tail(5)

In [128]:
school_summary_top_five_df

,School Name,School Type,Total School Budget,Total Students,Per student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
1,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,100.0,100.0,100.0
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,100.0,100.0,100.0
6,Holden High School,Charter,248087,427,581.0,83.803279,83.814988,100.0,100.0,100.0
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,100.0,100.0,100.0
11,Shelton High School,Charter,1056600,1761,600.0,83.359455,83.725724,100.0,100.0,100.0


In [129]:
school_summary_bottom_five_df

,School Name,School Type,Total School Budget,Total Students,Per student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
8,Johnson High School,District,3094650,4761,650.0,77.072464,80.966394,89.182945,100.0,94.591472
5,Hernandez High School,District,3022020,4635,652.0,77.289752,80.934412,89.083064,100.0,94.541532
7,Huang High School,District,1910635,2917,655.0,76.629414,81.182722,88.858416,100.0,94.429208
10,Rodriguez High School,District,2547363,3999,637.0,76.842711,80.744686,88.547137,100.0,94.273568
2,Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,88.436758,100.0,94.218379


In [130]:
school_grade_obj = student_school_df.groupby(by = ["school_name","grade"])

school_grade_math_df = school_grade_obj["math_score"].mean()
school_grade_math_df

school_name            grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

In [131]:
school_grade_reading_df = school_grade_obj["reading_score"].mean()
school_grade_reading_df

school_name            grade
Bailey High School     10th     80.907183
                       11th     80.945643
                       12th     80.912451
                       9th      81.303155
Cabrera High School    10th     84.253219
                       11th     83.788382
                       12th     84.287958
                       9th      83.676136
Figueroa High School   10th     81.408912
                       11th     80.640339
                       12th     81.384863
                       9th      81.198598
Ford High School       10th     81.262712
                       11th     80.403642
                       12th     80.662338
                       9th      80.632653
Griffin High School    10th     83.706897
                       11th     84.288089
                       12th     84.013699
                       9th      83.369193
Hernandez High School  10th     80.660147
                       11th     81.396140
                       12th     80.857143
     

In [132]:
bins = [550,580,610,640,670]
group_names = ['Low', 'Medium', 'High', 'Very High']
school_summary_df["Spending"] = pd.cut(school_summary_df["Per student Budget"], bins, labels=group_names)

average_spending_df=school_summary_df[["School Name","Spending","Average Math Score","Average Reading Score","% Passing Math",
                                      "% Passing Reading","Overall Passing"]]
average_spending_df

,School Name,Spending,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
1,Cabrera High School,Medium,83.061895,83.975780,100.000000,100.0,100.000000
4,Griffin High School,High,83.351499,83.816757,100.000000,100.0,100.000000
6,Holden High School,Medium,83.803279,83.814988,100.000000,100.0,100.000000
9,Pena High School,Medium,83.839917,84.044699,100.000000,100.0,100.000000
11,Shelton High School,Medium,83.359455,83.725724,100.000000,100.0,100.000000
12,Thomas High School,High,83.418349,83.848930,100.000000,100.0,100.000000
13,Wilson High School,Low,83.274201,83.989488,100.000000,100.0,100.000000
14,Wright High School,Medium,83.682222,83.955000,100.000000,100.0,100.000000
0,Bailey High School,High,77.048432,81.033963,89.529743,100.0,94.764871
3,Ford High School,Very High,77.102592,80.746258,89.302665,100.0,94.651333


In [133]:
def total_math_score(row):
    school = row["School Name"]
    value = student_school_df.loc[student_school_df['school_name'] == school, 'math_score'].sum()
    return value 

school_summary_df["Total Math Score"] = school_summary_df.apply(total_math_score,axis =1)

def total_reading_score(row):
    school = row["School Name"]
    value = student_school_df.loc[student_school_df['school_name'] == school, 'reading_score'].sum()
    return value 

school_summary_df["Total Reading Score"] = school_summary_df.apply(total_reading_score,axis =1)


In [134]:
spending_grp_obj = school_summary_df.groupby(by ="Spending")
spending_grp_math = (spending_grp_obj["Total Math Score"].sum())/(spending_grp_obj["Total Students"].sum())
spending_grp_math_df = pd.DataFrame(spending_grp_math)
spending_grp_math_df = spending_grp_math_df.reset_index()

spending_grp_reading = (spending_grp_obj["Total Reading Score"].sum())/(spending_grp_obj["Total Students"].sum())
spending_grp_reading_df = pd.DataFrame(spending_grp_reading)
spending_grp_reading_df = spending_grp_reading_df.reset_index()

grouped_spending_df = pd.merge(spending_grp_math_df,spending_grp_reading_df, on="Spending")
grouped_spending_df = grouped_spending_df.rename(columns ={"0_x":"Average Math Score"})
grouped_spending_df = grouped_spending_df.rename(columns ={"0_y":"Average Reading Score"})

spending_percent_reading = spending_grp_obj["% Passing Reading"].mean()
spending_percent_math = spending_grp_obj["% Passing Math"].mean()
spending_percent_math_df = pd.DataFrame(spending_percent_math)
spending_percent_math_df = spending_percent_math_df.reset_index()
spending_percent_reading_df = pd.DataFrame(spending_percent_reading)
spending_percent_reading_df = spending_percent_reading_df.reset_index()

grouped_spending_df = grouped_spending_df.merge(spending_percent_math_df,on="Spending")
grouped_spending_df = grouped_spending_df.merge(spending_percent_reading_df,on="Spending")


grouped_spending_df["Overall Rating"] = (grouped_spending_df["% Passing Math"] + grouped_spending_df["% Passing Reading"])/2
grouped_spending_df

,Spending,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Rating
0,Low,83.274201,83.989488,100.000000,100.0,100.000000
1,Medium,83.459313,83.905259,100.000000,100.0,100.000000
2,High,78.236441,81.559460,93.302728,100.0,96.651364
3,Very High,77.058995,80.958411,89.106772,100.0,94.553386


In [135]:
bins = [0,2000,3500,5000]
group_names = ['Small','Medium','Large']
school_summary_df["Size"] = pd.cut(school_summary_df["Total Students"], bins, labels=group_names)


In [136]:
size_group_obj = school_summary_df.groupby(by ="Size")

size_group_math = size_group_obj["Total Math Score"].sum()/size_group_obj["Total Students"].sum()
size_group_math_df = pd.DataFrame(size_group_math)
size_group_math_df = size_group_math_df.reset_index()


size_group_reading = size_group_obj["Total Reading Score"].sum()/size_group_obj["Total Students"].sum()
size_group_reading_df = pd.DataFrame(size_group_reading)
size_group_reading_df = size_group_reading_df.reset_index()

size_group_df = pd.merge(size_group_math_df,size_group_reading_df,on="Size")
size_group_df =size_group_df.rename(columns = {"0_x":"Average Math Score","0_y":"Average Reading Score"})


In [137]:
size_group_percent_math = size_group_obj["% Passing Math"].mean()
size_group_percent_math_df = pd.DataFrame(size_group_percent_math)
size_group_percent_math_df = size_group_percent_math_df.reset_index()

size_group_percent_reading = size_group_obj["% Passing Reading"].mean()
size_group_percent_reading_df = pd.DataFrame(size_group_percent_reading)
size_group_percent_reading_df = size_group_percent_reading_df.reset_index()

size_group_df = size_group_df.merge(size_group_percent_math_df,on= "Size")
size_group_df = size_group_df.merge(size_group_percent_reading_df,on= "Size")

size_group_df["Overall Rating"] =(size_group_df["% Passing Reading"] + size_group_df["% Passing Math"])/2

In [138]:
size_group_df

,Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Rating
0,Small,83.436586,83.882857,100.000000,100.0,100.000000
1,Medium,78.164034,81.654758,91.649460,100.0,95.824730
2,Large,77.070764,80.928365,89.085722,100.0,94.542861


In [139]:
type_group_obj = school_summary_df.groupby(by ="School Type")

type_group_math = type_group_obj["Total Math Score"].sum()/type_group_obj["Total Students"].sum()
type_group_math_df = pd.DataFrame(type_group_math)
type_group_math_df = type_group_math_df.reset_index()


type_group_reading = type_group_obj["Total Reading Score"].sum()/type_group_obj["Total Students"].sum()
type_group_reading_df = pd.DataFrame(type_group_reading)
type_group_reading_df = type_group_reading_df.reset_index()

type_group_df = pd.merge(type_group_math_df,type_group_reading_df,on="School Type")
type_group_df =type_group_df.rename(columns = {"0_x":"Average Math Score","0_y":"Average Reading Score"})

In [140]:
type_group_percent_math = type_group_obj["% Passing Math"].mean()
type_group_percent_math_df = pd.DataFrame(type_group_percent_math)
type_group_percent_math_df = type_group_percent_math_df.reset_index()

type_group_percent_reading = type_group_obj["% Passing Reading"].mean()
type_group_percent_reading_df = pd.DataFrame(type_group_percent_reading)
type_group_percent_reading_df = type_group_percent_reading_df.reset_index()

type_group_df = type_group_df.merge(type_group_percent_math_df,on= "School Type")
type_group_df = type_group_df.merge(type_group_percent_reading_df,on= "School Type")

type_group_df["Overall Rating"] =(type_group_df["% Passing Reading"] + type_group_df["% Passing Math"])/2

type_group_df

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Rating
0,Charter,83.406183,83.902821,100.000000,100.0,100.000000
1,District,76.987026,80.962485,88.991533,100.0,94.495766
